In [42]:
import os

os.chdir(r"C:\Users\hp\Documents\ML_Projects\Delhi_Infrastructure_Risk_AI")
print(os.getcwd())



C:\Users\hp\Documents\ML_Projects\Delhi_Infrastructure_Risk_AI


In [43]:
os.listdir()


['.venv',
 '.vscode',
 'app',
 'data',
 'DATA_SOURCES.md.txt',
 'gis',
 'models',
 'notebooks',
 'README.md.txt',
 'reports']

In [44]:
from pyrosm import OSM
import geopandas as gpd
from shapely.ops import unary_union
import os



In [45]:
boundary = gpd.read_file("gis/processed/delhi_ncr_districts.geojson")
boundary = boundary.to_crs(epsg=4326)



In [46]:
minx, miny, maxx, maxy = boundary.total_bounds

bbox = [minx, miny, maxx, maxy]
bbox


[np.float64(nan), np.float64(nan), np.float64(nan), np.float64(nan)]

In [47]:
bbox = [76.5, 28.2, 77.7, 29.0]   # Delhi NCR bbox


In [48]:
print(bbox)
print(type(bbox))
all(isinstance(x, (int, float)) for x in bbox)


[76.5, 28.2, 77.7, 29.0]
<class 'list'>


True

In [49]:
from pyrosm import OSM

osm = OSM(
    "gis/raw/northern-zone-260116.osm.pbf",
    bounding_box=bbox
)



In [50]:
roads = osm.get_network(network_type="driving")
roads.head()


c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\pyrosm\networks.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  edges, nodes = prepare_geodataframe(


,access,area,bicycle,bicycle_road,bridge,cycleway,est_width,foot,footway,highway,...,tracktype,tunnel,width,id,timestamp,version,tags,osm_type,geometry,length
0,None,None,None,None,None,None,None,None,None,trunk,...,None,None,None,5873630,1762774232,85,"{""visible"":false,""name:etymology:wikidata"":""Q1...",way,"MULTILINESTRING ((77.23161 28.6701, 77.23139 2...",1047.0
1,permissive,None,None,None,None,None,None,None,None,service,...,None,None,None,7892104,1681295429,3,"{""visible"":false}",way,"MULTILINESTRING ((77.16798 28.54259, 77.16793 ...",65.0
2,permissive,None,None,None,None,None,None,None,None,service,...,None,None,None,7892285,1681295429,17,"{""visible"":false}",way,"MULTILINESTRING ((77.16438 28.53842, 77.16449 ...",1014.0
3,permissive,None,None,None,None,None,None,None,None,service,...,None,None,None,7892369,1681295429,5,"{""visible"":false}",way,"MULTILINESTRING ((77.16643 28.54295, 77.16708 ...",175.0
4,permissive,None,None,None,None,None,None,None,None,service,...,None,None,None,7892539,1681295429,12,"{""visible"":false}",way,"MULTILINESTRING ((77.16209 28.54659, 77.16247 ...",538.0


In [51]:
roads = roads[
    ["name", "highway", "bridge", "tunnel", "length", "geometry"]
]


In [52]:
roads["is_flyover"] = roads["bridge"].notna()
roads["is_flyover"].value_counts()


is_flyover
False    277079
True       5051
Name: count, dtype: int64

In [58]:
roads.columns


Index(['name', 'highway', 'bridge', 'tunnel', 'length', 'is_flyover',
       'geometry'],
      dtype='object')

In [53]:
roads = gpd.overlay(roads, boundary, how="intersection")


In [59]:
roads = gpd.sjoin(
    roads,
    boundary[["geometry"]],
    how="inner",
    predicate="intersects"
)

roads = roads.drop(columns=["index_right"])


In [54]:
roads.to_file(
    "gis/processed/delhi_ncr_roads_flyovers.geojson",
    driver="GeoJSON"
)


In [57]:
import os
os.path.exists("gis/processed/delhi_ncr_roads_flyovers.geojson")


True